# Основы программирования в Python

*Автор: Алла Тамбовцева, НИУ ВШЭ*  

## Web-scraping: скрэйпинг новостного сайта

Мы уже немного познакомились со структурой html-файлов, теперь попробуем выгрузить информацию из реальной страницы, а точнее, с реального сайта [nplus1.ru](https://nplus1.ru/).

**Наша задача:** выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в csv-файл.

Сначала сгрузим весь html-код страницы и сохраним его в отдельную переменную. Для этого нам понадобится библиотека `requests`. Импортируем её:

In [1]:
import requests

Сохраним ссылку на главную страницу сайта в переменную `url` для удобства и выгрузим страницу. (Разумеется, это будет работать при подключении к интернету. Если соединение будет отключено, Python выдаст `NewConnectionError`).

In [2]:
url = 'https://nplus1.ru/' # сохраняем
page = requests.get(url) # загружаем страницу по ссылке

Если мы просто посмотрим на объект, мы ничего особенного не увидим:

In [3]:
page  # response 200 - страница загружена

<Response [200]>

Импортируем функцию `BeautifulSoup` из библиотеки `bs4` (от *beautifulsoup4*) и заберём со страницы `page` код html в виде текста. 

In [4]:
from bs4 import BeautifulSoup  # не спрашивайте, почему BeautifulSoup

In [5]:
soup = BeautifulSoup(page.text)

Если выведем `soup` на экран, мы увидим то же самое, что в режиме разработчика или в режиме происмотра исходного кода (`view-source` через *Ctrl+U* в Google Chrome).

In [6]:
# soup

Для просмотра выглядит не очень удобно.  «Причешем» наш `soup` – воспользуемся методом `.prettify()` в сочетании с функцией `print()`.

In [7]:
# print(soup.prettify())

В такой выдаче ориентироваться гораздо удобнее (но при желании, то же можно увидеть в браузере, на большом экране).

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Посмотрим на кусочки кода, соответствующие всем ссылкам на главной странице сайта:

In [8]:
for link in soup.find_all('a'):
    print(link.get('href'))

#
/
#
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
/theme/applying-statistics
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/theme/offline
/
#
/rubric/astronomy
/rubric/physics
/rubric/biology
/rubric/robots-drones
#
/theme/applying-statistics
/theme/bookshelf
/theme/Courses
/theme/coronavirus-history
/theme/offline
https://nplus1.ru/blog/2021/12/01/physics-problems
https://nplus1.ru/blog/2021/12/01/physics-problems
https://nplus1.ru/blog/2021/11/30/into-the-abyss
https://nplus1.ru/blog/2021/11/26/a-woman-s-heart
https://nplus1.ru/blog/2021/11/25/deep-medicine
https://nplus1.ru/blog/2021/11/23/the-russian-job
https://nplus1.ru/blog/2021/11/11/a-surgeons-notes-on-performance
https://nplus1.ru/blog/2021/11/02/rudalle
https://nplus1.ru/blog/2021/11/03/dune-climate
https://nplus1.ru/blog/2021/11/03/tales-from-both-sides-of-the-brain
https://nplus1.ru/blog/2021/12/02/useful-delusions
/news/2021/12/03/b62-12
/news/2021/12/03/quantum-OPM-MEG
/news/202

Ссылок много. Но нам нужны только новости – ссылки, которые начинаются со слова `/news`. Добавим условие: будем выбирать только те ссылки, в которых есть `/news`. Создадим пустой список `urls` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [9]:
urls = []

for link in soup.find_all('a'):
    if '/news' in link.get('href'):
        urls.append(link.get('href'))
urls

['/news/2021/12/03/b62-12',
 '/news/2021/12/03/quantum-OPM-MEG',
 '/news/2021/12/03/qualcomm-always-on',
 '/news/2021/12/03/e-coli-inf',
 '/news/2021/12/03/cabless',
 '/news/2021/12/03/japanmissiles',
 '/news/2021/12/03/tenet-of-sound',
 '/news/2021/12/03/adenovirus-thrombosis',
 '/news/2021/12/03/gj-367-b',
 '/news/2021/12/03/stingray',
 '/news/2021/12/03/equids-stone-artefacts',
 '/news/2021/12/03/bronze-age-hoards',
 '/news/2021/12/03/lonesome-otter',
 '/news/2021/12/03/stegouros-elengassen',
 '/news/2021/12/03/neutron-rocket',
 '/news/2021/12/02/activity-of-cumbre-vieja',
 '/news/2021/12/02/drone-perching',
 '/news/2021/12/02/lead-whites',
 '/news/2021/12/02/surprise-fm',
 '/news/2021/11/30/skyrmion-light-beam',
 '/news/2021/11/25/heavy-fermions',
 '/news/2021/11/24/e-v-reconstruction',
 '/news/2021/11/26/solid-state-nmr-statistics',
 '/news/2021/11/24/LOV',
 '/news/2021/12/02/surprise-fm',
 '/news/2021/11/29/LA-Moire',
 '/news/2021/11/26/relativity-simulations',
 '/news/2021/11/24

Ссылки, которые у нас есть в списке `urls`, относительные: они неполные, начало ссылки (название сайта) отсутствует. Давайте превратим их в абсолютные ‒ склеим с ссылкой https://nplus1.ru.

In [10]:
full_urls = []

for u in urls:
    res = 'https://nplus1.ru' + u
    full_urls.append(res) 

full_urls

['https://nplus1.ru/news/2021/12/03/b62-12',
 'https://nplus1.ru/news/2021/12/03/quantum-OPM-MEG',
 'https://nplus1.ru/news/2021/12/03/qualcomm-always-on',
 'https://nplus1.ru/news/2021/12/03/e-coli-inf',
 'https://nplus1.ru/news/2021/12/03/cabless',
 'https://nplus1.ru/news/2021/12/03/japanmissiles',
 'https://nplus1.ru/news/2021/12/03/tenet-of-sound',
 'https://nplus1.ru/news/2021/12/03/adenovirus-thrombosis',
 'https://nplus1.ru/news/2021/12/03/gj-367-b',
 'https://nplus1.ru/news/2021/12/03/stingray',
 'https://nplus1.ru/news/2021/12/03/equids-stone-artefacts',
 'https://nplus1.ru/news/2021/12/03/bronze-age-hoards',
 'https://nplus1.ru/news/2021/12/03/lonesome-otter',
 'https://nplus1.ru/news/2021/12/03/stegouros-elengassen',
 'https://nplus1.ru/news/2021/12/03/neutron-rocket',
 'https://nplus1.ru/news/2021/12/02/activity-of-cumbre-vieja',
 'https://nplus1.ru/news/2021/12/02/drone-perching',
 'https://nplus1.ru/news/2021/12/02/lead-whites',
 'https://nplus1.ru/news/2021/12/02/surpri

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `full_urls` в цикле. Посмотрим на новость с индексом 1, у вас может быть другая, новости обновляются.

In [11]:
url0 = full_urls[1]

page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text)

В коде каждой страницы с новостью есть часть с мета-информацией: датой, именем автора и проч. Такая информация окружена тэгом `<meta></meta>`. Посмотрим:

In [12]:
soup0.find_all('meta')

[<meta charset="utf-8"/>,
 <meta content="ie=edge" http-equiv="x-ua-compatible"/>,
 <meta content="width=device-width, initial-scale=1" name="viewport"/>,
 <meta content="yes" name="apple-mobile-web-app-capable"/>,
 <meta content="black" name="apple-mobile-web-app-status-bar-style"/>,
 <meta content="7991d7eb02d759f05b9050e111a7e3eb" name="wmail-verification"/>,
 <meta content="2021-12-03" itemprop="datePublished"/>,
 <meta content="Оксана Борзенкова" name="mediator_author"/>,
 <meta content="и сделать их удобнее в использовании" name="description"/>,
 <meta content="Оксана Борзенкова" name="author"/>,
 <meta content="" name="copyright"/>,
 <meta content="Магнитометры улучшили качество и скорость нейровизуализации" property="og:title"/>,
 <meta content="https://nplus1.ru/images/2021/12/02/bd8a26338c953792b2bac063e77fe5e4.jpg" property="og:image"/>,
 <meta content="https://nplus1.ru/news/2021/12/03/quantum-OPM-MEG" property="og:url"/>,
 <meta content="и сделать их удобнее в использовани

Из этого списка нам нужны части с именем автора, датой, заголовком и кратким описанием. Воспользуемся поиском по атрибуту `name`. Передадим функции `find_all()` в качестве аргумента словарь с названием и значением атрибута: 

In [13]:
soup0.find_all('meta', {'name' : 'author'}) # например, автор

[<meta content="Оксана Борзенкова" name="author"/>]

Теперь выберем единственный элемент полученного списка (с индексом 0):

In [14]:
soup0.find_all('meta', {'name' : 'author'})[0]

<meta content="Оксана Борзенкова" name="author"/>

Нам нужно вытащить из этого объекта `content` – имя автора. Посмотрим на атрибуты:

In [15]:
soup0.find_all('meta', {'name' : 'author'})[0].attrs

{'name': 'author', 'content': 'Оксана Борзенкова'}

Как получить отсюда `content`? Очень просто, ведь это словарь! А доставать из словаря значение по ключу мы умеем.

In [16]:
author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
author

'Оксана Борзенкова'

Аналогичным образом извлечем дату, заголовок и описание.

In [17]:
date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']

Осталось вытащить рубрики и сложность текста. Если мы посмотрим на исходный код страницы, мы увидим, что нужная нам информация находится в тэгах `<p></p>`:

In [18]:
soup0.find_all('p')

[<p class="table">
 <a data-rubric="physics" href="/rubric/physics">Физика</a>
 <a data-rubric="biology" href="/rubric/biology">Биология</a>
 </p>,
 <p class="table">
 <a href="/news/2021/12/03">
 <time content="2021-12-03" data-unix="1638544634" itemprop="datePublished">
 <span>18:17</span>
 <span>03 Дек. 2021</span>
 </time>
 </a>
 </p>,
 <p class="table">
 <a href="/difficult/6.6">
 <span>Сложность</span>
 <span class="difficult-value">6.6</span>
 </a>
 </p>,
 <p class="title"></p>,
 <p class="credits">Aikaterini Gialopsou et al. / Nature, 2021</p>,
 <p>Ученые показали, как можно усовершенствовать существующие методы нейровизуализации с помощью атомного магнитометра с оптической накачкой. Благодаря высокой чувствительности и использованию оптических лучей, такое устройство позволяет с лучшим разрешением детектировать сигналы мозга и оказывается более удобным в применении. Работа <a href="https://www.nature.com/articles/s41598-021-01854-7" rel="nofollow" target="_blank">опубликована<

Выберем из полученного списка первый элемент и найдем в нем все тэги `<a>`:

In [19]:
soup0.find_all('p')[0].find_all('a')

[<a data-rubric="physics" href="/rubric/physics">Физика</a>,
 <a data-rubric="biology" href="/rubric/biology">Биология</a>]

Получился список из одного элемента. Применим списковые включения – вытащим из каждого элемента (их могло бы быть больше) текст и поместим его в новый список `rubrics`.

In [20]:
rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
rubrics

['Физика', 'Биология']

Осталась только сложность. Возьмем соответствующий кусок кода:

In [21]:
soup0.find_all('span', {'class' : 'difficult-value'})

[<span class="difficult-value">6.6</span>]

Теперь осталось совсем чуть-чуть. Написать готовую функцию для всех проделанных нами действий и применить ее в цикле для всех ссылок в списке `full_urls`. Напишем! Аргументом функции будет ссылка на новость, а возвращать она будет текст новости и всю необходимую информацию (дата, автор, сложность и проч.). Скопируем все строки кода выше.

In [22]:
def GetNews(url0):
    """
    Returns a tuple with url0, date, author, description, title, final_text, rubrics, diff.
    Parameters:
    
    url0 is a link to the news (string)
    """
    page0 = requests.get(url0)
    soup0 = BeautifulSoup(page0.text)
    author = soup0.find_all('meta', {'name' : 'author'})[0].attrs['content']
    date = soup0.find_all('meta', {'itemprop' : 'datePublished'})[0].attrs['content']
    title = soup0.find_all('meta', {'property' : 'og:title'})[0].attrs['content']
    description = soup0.find_all('meta', {'name' : 'description'})[0].attrs['content']
    rubrics = [r.text for r in soup0.find_all('p')[0].find_all('a')]
    diff = soup0.find_all('span', {'class' : 'difficult-value'})[0].text
    
    return url0, date, author, description, title, rubrics, diff

Уфф. Осталось применить ее в цикле. Но давайте не будем спешить: импортируем функцию `sleep` для задержки, чтобы на каждой итерации цикла, прежде чем перейти к следующей новости, Python ждал несколько секунд. Во-первых, это нужно, чтобы сайт «не понял», чтобы мы его грабим, да еще автоматически. Во-вторых, с небольшой задержкой всегда есть гарантия, что страница прогрузится (сейчас это пока не очень важно, но особенно актуально будет, когда будем обсуждать встраивание в браузер с Selenium). Приступим.

In [23]:
from time import sleep

In [24]:
news = [] # это будет список из кортежей, в которых будут храниться данные по каждой новости

for link in full_urls:
    res = GetNews(link)
    news.append(res)
    sleep(3) # задержка в 3 секунды

Так теперь выглядит первый элемент списка:

In [25]:
news[0]

('https://nplus1.ru/news/2021/12/03/b62-12',
 '2021-12-03',
 'Василиса Чернявцева',
 'Полномасштабное серийное производство боеприпаса начнется в мае 2022 года и завершится в 2026 финансовом году',
 'Американцы собрали первую серийную модернизированную термоядерную бомбу',
 ['Оружие'],
 '2.8')

Импортируем `pandas` и создадим датафрейм из списка кортежей: 

In [26]:
import pandas as pd

In [27]:
df = pd.DataFrame(news)

In [28]:
df.head(2)

,0,1,2,3,4,5,6
0,https://nplus1.ru/news/2021/12/03/b62-12,2021-12-03,Василиса Чернявцева,Полномасштабное серийное производство боеприпа...,Американцы собрали первую серийную модернизиро...,[Оружие],2.8
1,https://nplus1.ru/news/2021/12/03/quantum-OPM-MEG,2021-12-03,Оксана Борзенкова,и сделать их удобнее в использовании,Магнитометры улучшили качество и скорость нейр...,"[Физика, Биология]",6.6


Переименуем столбцы в базе.

In [30]:
df.columns = ['link', 'date', 'author', 'desc', 'title', 'rubric', 'diffc']

In [31]:
df.head(2)

,link,date,author,desc,title,rubric,diffc
0,https://nplus1.ru/news/2021/12/03/b62-12,2021-12-03,Василиса Чернявцева,Полномасштабное серийное производство боеприпа...,Американцы собрали первую серийную модернизиро...,[Оружие],2.8
1,https://nplus1.ru/news/2021/12/03/quantum-OPM-MEG,2021-12-03,Оксана Борзенкова,и сделать их удобнее в использовании,Магнитометры улучшили качество и скорость нейр...,"[Физика, Биология]",6.6


Теперь внесем изменения: сделаем столбец `diffc` числовым – типа *float*.

In [32]:
df.to_excel('nplus-news.xlsx')